In [7]:
import torch
from torch.utils.data import DataLoader
import os
import pandas as pd
from utils.dataset import PandasDataset
from utils.models import EfficientNet
from utils.metrics import evaluation, format_metrics
import albumentations

In [8]:
output_dimensions = 5
data_dir = '../../../dataset'
ROOT_DIR = "../../"
images_dir = os.path.join(data_dir, 'tiles')
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
df_test = pd.read_csv(f"../../data/test.csv")
backbone_model = 'efficientnet-b0'
pretrained_model = {
    backbone_model: f'{ROOT_DIR}/efficientnet-b0-08094119.pth'
}

In [9]:
print("Cuda", device)

Cuda cuda


In [10]:
dataloader = DataLoader(
    PandasDataset(images_dir, df_test, transforms=None, normalize=True),
    batch_size=2,
    shuffle=False,
)

In [11]:
model = EfficientNet(
    backbone=backbone_model,
    output_dimensions=output_dimensions,
    pre_trained_model=pretrained_model
)
model.to(device).load_state_dict(
    torch.load(
        f"{ROOT_DIR}/models/with-noise-rgb.pth",
        weights_only=True,
        map_location=device
    )
)

Loaded pretrained weights for efficientnet-b0


<All keys matched successfully>

In [12]:
response = evaluation(model, dataloader, device)
result = format_metrics(response[0])
print(result)

100%|██████████| 796/796 [01:41<00:00,  7.82it/s]


VAL_ACC      Mean: 12.60 | Std: 0.84 | 95% CI: [11.24, 14.07]
VAL_KAPPA    Mean: 0.00 | Std: 0.00 | 95% CI: [0.00, 0.00]
VAL_F1       Mean: 0.04 | Std: 0.00 | 95% CI: [0.03, 0.04]
VAL_RECALL   Mean: 0.17 | Std: 0.00 | 95% CI: [0.17, 0.17]
VAL_PRECISION Mean: 0.02 | Std: 0.00 | 95% CI: [0.02, 0.02]
